In [1]:
!pip install catboost

     |████████████████████████████████| 67.3MB 54kB/s 


In [2]:
!pip install nltk

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from sklearn.metrics import roc_auc_score
import numpy as np

from catboost import CatBoostClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


# Datasets preprocessing

In [147]:
with open("dataset_ok.txt", "r+") as file:
  file_readen = file.read().split("\n")

In [148]:
labels = list(map(lambda x: list(map(lambda y: y[9 :], x.split(" ")[0].split(","))), file_readen))[: -1]
text = list(map(lambda x: " ".join(x.split(" ")[1 :]), file_readen))[: -1]

In [149]:
df = pd.DataFrame(np.array([labels, text]).T, columns = ["label", "text"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [150]:
def onehot(series, unique):
  series_new = pd.DataFrame(data = np.array([[0 for j in range(len(series))] for i in range(len(unique))]).T ,columns = unique)
  for elIndex in range(len(series)):
      for el_class in series.iloc[elIndex]: series_new[el_class].iloc[elIndex] = 1
  return series_new

In [151]:
df_onehoted = onehot(df.label, ['NORMAL', 'INSULT', 'THREAT', 'OBSCENITY'])

In [152]:
df_onehoted.INSULT.value_counts()

0    211464
1     36826
Name: INSULT, dtype: int64

In [153]:
df_onehoted.THREAT.value_counts()

0    236263
1     12027
Name: THREAT, dtype: int64

In [154]:
df_onehoted.OBSCENITY.value_counts()

0    244029
1      4261
Name: OBSCENITY, dtype: int64

In [155]:
df_onehoted

,NORMAL,INSULT,THREAT,OBSCENITY
0,0,1,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
248285,1,0,0,0
248286,0,1,0,0
248287,1,0,0,0
248288,1,0,0,0


In [156]:
df_2ch = pd.read_csv("labeled.csv")
df_2ch['toxic'] = df_2ch['toxic'].astype(int)

In [157]:
df_2ch

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1


In [158]:
# snowball = SnowballStemmer(language="russian")
# nltk.download('stopwords')
# russian_stop_words = stopwords.words("russian")

# def tokenize_sentence(sentence: str, remove_stop_words: bool = True) -> list:
#     tokens = word_tokenize(sentence, language="russian")
#     tokens = [i for i in tokens if i not in string.punctuation]
#     if remove_stop_words:
#         tokens = [i for i in tokens if i not in russian_stop_words]
#     tokens = [snowball.stem(i) for i in tokens]
#     return tokens

In [159]:
X_train_ins, X_test_ins, y_train_ins, y_test_ins = train_test_split(df.text, df_onehoted.INSULT, test_size=0.33, random_state=42)
X_train_thr, X_test_thr, y_train_thr, y_test_thr = train_test_split(df.text, df_onehoted.THREAT, test_size=0.33, random_state=42)
X_train_obs, X_test_obs, y_train_obs, y_test_obs = train_test_split(df.text, df_onehoted.OBSCENITY, test_size=0.33, random_state=42)

# Sentences vectorizing

In [20]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=c8aff3ba4657f660315d4cb97c5070f596ce07fdd1ece2b2894e572dca077efc
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [79]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-06-02 12:13:01--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.243, 5.45.205.244, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.243|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm905.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-06-02 12:13:01--  http://cache-mskm905.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm905.cdn.yandex.net (cache-mskm905.cdn.yandex.net)... 5.45.220.15, 2a02:6b8:0:2002::16
Connecting to cache-mskm905.cdn.yandex.net (cache-mskm905.cdn.yandex.net)|5.45.220.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[===================>] 

In [194]:
import wget, zipfile, gensim

import numpy as np
import pandas as pd

from functools import lru_cache
from pymystem3 import Mystem

class Vectorizer:
  def __init__(self, model_file: str = ""):
    self.mapping = {'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ', 'APRO': 'DET', 'COM': 'ADJ', 'CONJ': 'SCONJ', 'INTJ': 'INTJ', 'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART', 'PR': 'ADP', 'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}

    self.mystem = Mystem()

    self.model_vv = None

    if (len(model_file) == 0): self.download_model()
    else: self.load_model(model_file)
  
  def download_model(self):
    # udpipe url: https://rusvectores.org/static/models/udpipe_syntagrus.model
    model_url = 'http://vectors.nlpl.eu/repository/20/204.zip'

    m_ = wget.download(model_url)

    model_file = model_url.split('/')[-1]

    print(f"Model downloaded from {model_url}\n Path: {model_file}")

    self.load_model(model_file)

  def load_model(self, model_file: str):
    with zipfile.ZipFile(model_file, 'r') as archive:
      stream = archive.open('model.bin')
      self.model_vv = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

    print("Model loaded.")

  @lru_cache(maxsize=None)
  def tag_mystem(self, text: str) -> list:
    try:
      processed = self.mystem.analyze(text)
    except:
      return []
    tagged = []

    for w in processed:
      try:
        lemma = w["analysis"][0]["lex"].lower().strip()
        pos = w["analysis"][0]["gr"].split(',')[0]
        pos = pos.split('=')[0].strip()
        if pos in self.mapping:
            tagged.append(lemma + '_' + self.mapping[pos])
        else:
            tagged.append(lemma + '_X')
      except KeyError:
        continue
      except IndexError:
        continue
    return tagged
  
  def tokenize_word2vec(self, sentence: str) -> list:
    tokens = self.tag_mystem(sentence)
    vectors = []

    for token in tokens:
      try:
        vectors.append(self.model_vv[token])
      except:
        continue

    if (len(vectors) == 0): return self.model_vv["слэнг_NOUN"]

    return np.mean(vectors, axis = 0)
  
  def Vectorize_one(self, text: str) -> np.array:
    self.mystem = Mystem()
    return self.tokenize_word2vec(text)

  def Vectorize_corpus(self, dataset: pd.DataFrame) -> pd.DataFrame:
    self.mystem = Mystem()
    
    vectorized = []

    for object_index in range(len(dataset)):
      if (object_index % 10000 == 0): print(f"Current progress: {object_index / len(dataset) * 100}%")
      
      vector = self.tokenize_word2vec(dataset.iloc[object_index])

      vectorized.append(vector)
    
    return pd.DataFrame(list(vectorized))

In [195]:
vectorizer = Vectorizer(model_file="204.zip")

Model loaded.


In [160]:
X_train_vec = vectorizer.Vectorize_corpus(X_train_ins)

Current progress: 0.0%
Current progress: 6.011277155944552%
Current progress: 12.022554311889104%
Current progress: 18.033831467833654%
Current progress: 24.04510862377821%
Current progress: 30.05638577972276%
Current progress: 36.06766293566731%
Current progress: 42.07894009161187%
Current progress: 48.09021724755642%
Current progress: 54.10149440350097%
Current progress: 60.11277155944552%
Current progress: 66.12404871539007%
Current progress: 72.13532587133462%
Current progress: 78.14660302727917%
Current progress: 84.15788018322374%
Current progress: 90.16915733916828%
Current progress: 96.18043449511283%


In [161]:
X_test_vec = vectorizer.Vectorize_corpus(X_test_ins)

Current progress: 0.0%
Current progress: 12.20464752977934%
Current progress: 24.40929505955868%
Current progress: 36.61394258933802%
Current progress: 48.81859011911736%
Current progress: 61.023237648896696%
Current progress: 73.22788517867605%
Current progress: 85.43253270845538%
Current progress: 97.63718023823472%


# Train

## Logistic Regression

In [88]:
model_lr = LogisticRegression(random_state=42)

In [99]:
model_lr.fit(X_train_vec, y_train_ins)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [100]:
precision_score(y_true=y_test_ins, y_pred=model_lr.predict(X_test_ins_vec))

0.8058740037121956

In [101]:
recall_score(y_true=y_test_ins, y_pred=model_lr.predict(X_test_ins_vec))

0.6029736132668899

In [102]:
roc_auc_score(y_test_ins, model_lr.predict_proba(X_test_ins_vec).T[1])

0.9478907815331472

## CatBoost

In [162]:
model_ins_cbc = CatBoostClassifier(random_state=42)
model_thr_cbc = CatBoostClassifier(random_state=42)
model_obs_cbc = CatBoostClassifier(random_state=42)

In [178]:
model_ins_cbc.fit(X_train_vec, y_train_ins)

Learning rate set to 0.091478
0:	learn: 0.5934829	total: 354ms	remaining: 5m 53s
1:	learn: 0.5210373	total: 727ms	remaining: 6m 2s
2:	learn: 0.4638819	total: 1.08s	remaining: 6m
3:	learn: 0.4204441	total: 1.44s	remaining: 5m 59s
4:	learn: 0.3848853	total: 1.84s	remaining: 6m 5s
5:	learn: 0.3571707	total: 2.21s	remaining: 6m 5s
6:	learn: 0.3340549	total: 2.55s	remaining: 6m 1s
7:	learn: 0.3166345	total: 2.89s	remaining: 5m 58s
8:	learn: 0.3029785	total: 3.23s	remaining: 5m 55s
9:	learn: 0.2918002	total: 3.56s	remaining: 5m 52s
10:	learn: 0.2819203	total: 3.9s	remaining: 5m 50s
11:	learn: 0.2733237	total: 4.23s	remaining: 5m 48s
12:	learn: 0.2656285	total: 4.57s	remaining: 5m 46s
13:	learn: 0.2599591	total: 4.91s	remaining: 5m 45s
14:	learn: 0.2542522	total: 5.25s	remaining: 5m 44s
15:	learn: 0.2494366	total: 5.58s	remaining: 5m 42s
16:	learn: 0.2448126	total: 5.94s	remaining: 5m 43s
17:	learn: 0.2408042	total: 6.29s	remaining: 5m 43s
18:	learn: 0.2371053	total: 6.63s	remaining: 5m 42s
1

In [165]:
model_thr_cbc.fit(X_train_vec, y_train_thr)

Learning rate set to 0.091478
0:	learn: 0.5508853	total: 442ms	remaining: 7m 21s
1:	learn: 0.4477574	total: 781ms	remaining: 6m 29s
2:	learn: 0.3673323	total: 1.12s	remaining: 6m 10s
3:	learn: 0.3070960	total: 1.49s	remaining: 6m 10s
4:	learn: 0.2615937	total: 1.87s	remaining: 6m 12s
5:	learn: 0.2298554	total: 2.22s	remaining: 6m 7s
6:	learn: 0.2033020	total: 2.58s	remaining: 6m 6s
7:	learn: 0.1863568	total: 3.01s	remaining: 6m 13s
8:	learn: 0.1716922	total: 3.56s	remaining: 6m 31s
9:	learn: 0.1588857	total: 4.04s	remaining: 6m 39s
10:	learn: 0.1493104	total: 4.5s	remaining: 6m 44s
11:	learn: 0.1420294	total: 5.02s	remaining: 6m 52s
12:	learn: 0.1360267	total: 5.42s	remaining: 6m 51s
13:	learn: 0.1309703	total: 5.79s	remaining: 6m 48s
14:	learn: 0.1275420	total: 6.15s	remaining: 6m 43s
15:	learn: 0.1243413	total: 6.54s	remaining: 6m 42s
16:	learn: 0.1210523	total: 6.91s	remaining: 6m 39s
17:	learn: 0.1183524	total: 7.3s	remaining: 6m 38s
18:	learn: 0.1162447	total: 7.68s	remaining: 6m 

In [167]:
model_obs_cbc.fit(X_train_vec, y_train_obs)

Learning rate set to 0.091478
0:	learn: 0.5247640	total: 309ms	remaining: 5m 8s
1:	learn: 0.3950339	total: 616ms	remaining: 5m 7s
2:	learn: 0.3079338	total: 903ms	remaining: 5m
3:	learn: 0.2392337	total: 1.19s	remaining: 4m 55s
4:	learn: 0.1977978	total: 1.48s	remaining: 4m 54s
5:	learn: 0.1630252	total: 1.76s	remaining: 4m 51s
6:	learn: 0.1334197	total: 2.1s	remaining: 4m 58s
7:	learn: 0.1151828	total: 2.4s	remaining: 4m 57s
8:	learn: 0.1011840	total: 2.71s	remaining: 4m 58s
9:	learn: 0.0901804	total: 3.01s	remaining: 4m 57s
10:	learn: 0.0830463	total: 3.33s	remaining: 4m 59s
11:	learn: 0.0771429	total: 3.64s	remaining: 5m
12:	learn: 0.0722148	total: 4s	remaining: 5m 3s
13:	learn: 0.0680282	total: 4.36s	remaining: 5m 7s
14:	learn: 0.0648872	total: 4.69s	remaining: 5m 7s
15:	learn: 0.0628241	total: 5.01s	remaining: 5m 8s
16:	learn: 0.0606174	total: 5.36s	remaining: 5m 10s
17:	learn: 0.0587314	total: 5.69s	remaining: 5m 10s
18:	learn: 0.0571197	total: 6.02s	remaining: 5m 10s
19:	learn: 

## Models with old vectorization

In [ ]:
models_pipeline = {
    "INS": Pipeline([("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence)), ("model", LogisticRegression(random_state=42, C=10))]),
    "THR": Pipeline([("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence)), ("model", LogisticRegression(random_state=42, C=10))]), 
    "OBS": Pipeline([("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence)), ("model", LogisticRegression(random_state=42, C=10))])
    }

In [ ]:
models_pipeline['INS'].fit(X_train_ins, y_train_ins)

In [ ]:
models_pipeline['THR'].fit(X_train_thr, y_train_thr)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...w+\\b',
                                 tokenizer=<function tokenize_sentence at 0x7fa6bdfcac20>,
                                 use_idf=True, vocabulary=None)),
                ('model',
                 LogisticRegression(C=10, class_weight=None, dual=False,
                                    fit

In [ ]:
models_pipeline['OBS'].fit(X_train_obs, y_train_obs)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...w+\\b',
                                 tokenizer=<function tokenize_sentence at 0x7fa6bdfcac20>,
                                 use_idf=True, vocabulary=None)),
                ('model',
                 LogisticRegression(C=10, class_weight=None, dual=False,
                                    fit

# Score&Results

## Old models

In [ ]:
print("Precision: {}; Recall: {}".format(precision_score(y_true=y_test_ins, y_pred=models_pipeline['INS'].predict(X_test_ins), average = 'weighted'), recall_score(y_true=y_test_ins, y_pred=models_pipeline['INS'].predict(X_test_ins), average = 'weighted')))

Precision: 0.9599445125554553; Recall: 0.960676625659051


In [ ]:
print("Precision: {}; Recall: {}".format(precision_score(y_true=y_test_thr, y_pred=models_pipeline['THR'].predict(X_test_thr), average = 'weighted'), recall_score(y_true=y_test_thr, y_pred=models_pipeline['THR'].predict(X_test_thr), average = 'weighted')))

Precision: 0.9774182679964719; Recall: 0.9786784807654755


In [ ]:
print("Precision: {}; Recall: {}".format(precision_score(y_true=y_test_obs, y_pred=models_pipeline['OBS'].predict(X_test_obs), average = 'weighted'), recall_score(y_true=y_test_obs, y_pred=models_pipeline['OBS'].predict(X_test_obs), average = 'weighted')))

Precision: 0.9913580079452491; Recall: 0.9919693419254052


In [ ]:
roc_auc_score(models_pipeline["INS"].predict(df_2ch.comment), df_2ch.toxic)

0.8399818766217839

In [ ]:
df_2ch.iloc[np.where(models_pipeline["THR"].predict(df_2ch.comment) == 1)]

,comment,toxic
2,Собаке - собачья смерть\n,1
97,Что б он сам здох мразь\n,1
117,Таких как ты психопатов на хуй отстреливать в ...,1
142,Вы привели достаточно аргументов чтобы их не п...,1
348,стрелять в орущее быдло\n,1
...,...,...
14003,Пиар параши. Сделайте этому хуесосу уже гребеш...,1
14107,Это моча так неугодных гнобит. Даёт бан педрюх...,1
14146,"И кто же тебя обоссал на этот раз, россиянец?\n",1
14388,ПРОКЛЯТАЯ ДЕТОЕДСКАЯ МРАЗЬ. ГОРИ В АДУ ВЕЧНО.\n,1


In [ ]:
df_2ch.iloc[np.where(models_pipeline["OBS"].predict(df_2ch.comment) == 1)]

,comment,toxic
163,"о жопах Да нет, это твой свидомый мозг дорисов...",1
274,"мне поебать на его финансы и кто он. Не пизди,...",1
373,Я атеист и армянин. Просто рот твой ебал Лучше...,1
374,"Конечно, вы же собак ебете, блядь.\n",1
777,"Я ШЛЮХА -- ТЕТЯ СРАКА, ВАША ЗАРПЛАТА ПОНИЖЕНА....",1
...,...,...
14195,"Тебе уже предоставляется такая возможность, бе...",1
14231,не захотел отсасывать мракобесию И с заглотом ...,1
14232,Соси хуй подсос единоросский\n,1
14258,"Хуй соси, анимеблядь. Мать твою ебал Аноним ID...",1


In [ ]:
model_pipeline[0].predict(["всем привет"])

array(['NORMAL'], dtype=object)

## New models

In [197]:
precision_score(y_true=y_test_ins, y_pred=model_ins_cbc.predict(X_test_vec)), precision_score(y_true=y_test_thr, y_pred=model_thr_cbc.predict(X_test_vec)), precision_score(y_true=y_test_obs, y_pred=model_obs_cbc.predict(X_test_vec))

(0.8709055876685935, 0.8530405405405406, 0.8199566160520607)

In [198]:
recall_score(y_true=y_test_ins, y_pred=model_ins_cbc.predict(X_test_vec)), recall_score(y_true=y_test_thr, y_pred=model_thr_cbc.predict(X_test_vec)), recall_score(y_true=y_test_obs, y_pred=model_obs_cbc.predict(X_test_vec))

(0.7385017563924516, 0.625619425173439, 0.5470332850940666)

In [199]:
roc_auc_score(y_test_ins, model_ins_cbc.predict_proba(X_test_vec).T[1]), roc_auc_score(y_test_thr, model_thr_cbc.predict_proba(X_test_vec).T[1]), roc_auc_score(y_test_obs, model_obs_cbc.predict_proba(X_test_vec).T[1])

(0.9670210532958152, 0.9796235421641681, 0.976706747165172)

In [118]:
df_2ch_vec = vectorizer.Vectorize(dataset=df_2ch.comment)

Current progress: 0.0%
Current progress: 69.38662225922843%


### Insults

In [183]:
df_2ch_toxic_predicted = df_2ch.iloc[np.where(model_ins_cbc.predict(df_2ch_vec) == 1)]
df_2ch_untoxic_predicted = df_2ch.iloc[np.where(model_ins_cbc.predict(df_2ch_vec) == 0)]

False Positive predictions

In [184]:
df_2ch_toxic_predicted[df_2ch_toxic_predicted.toxic == 0]

,comment,toxic
109,Да нахуй ты мне это скинул? Если я теку от неё...,0
131,Камера заднего вида и книжная лига? Вы тролль\n,0
172,"Конечно, это резонансное событие требует отдел...",0
178,"Хм... По-моему, ты не слышал, как кричит такой...",0
233,"Качок дуболом это я так понимаю голубочкин, бл...",0
...,...,...
14013,"А вот говорить что сниматься в порно хорошо, н...",0
14028,Это хуйло хоть 1 полноценный мульт нарисовало?...,0
14254,Крупные производители уже давно напрочь охуели...,0
14281,Ну давайте тогда всё рассуём по тематике: Танц...,0


True positive predictions

In [185]:
df_2ch_toxic_predicted[df_2ch_toxic_predicted.toxic == 1]

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
5,Для каких стан является эталоном современная с...,1
8,"Ебать тебя разносит, шизик.\n",1
...,...,...
14400,"Не зря, вас, хохлов, свиньями кличут. Вы и ест...",1
14402,"ЭЙ МИХАЛЫЧ, ТАМ ПОЖАРКА ПИЩИТ ДА ХУЙ С НЕЙ ПУЩ...",1
14403,Пусть лучше само евровидение стримит. Долбоеб ...,1
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1


False Negative predictions

In [186]:
df_2ch_untoxic_predicted[df_2ch_untoxic_predicted.toxic == 1]

,comment,toxic
2,Собаке - собачья смерть\n,1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1
7,УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РА...,1
11,"Бактерия, тебе этого не понять Конечно, я же н...",1
15,"хорош врать, ты террорист-торчёк-шизофреник пр...",1
...,...,...
14398,Моча в анимаче решила окончательно убить реакш...,1
14401,Ну не прямо сразу же ватан бросился к своему к...,1
14404,"ЖЕНЩИНА это ВЕНЕЦ ТВОРЕНИЯ, помните это ваньки...",1
14405,Сейчас все авиакомпании мира вместе со специал...,1


True Negative predictions

In [187]:
df_2ch_untoxic_predicted[df_2ch_untoxic_predicted.toxic == 0]

,comment,toxic
6,В шапке были ссылки на инфу по текущему фильму...,0
12,"Почитайте посты у этого автора,может найдете ч...",0
17,Про графику было обидно) я так то проходил все...,0
28,https: pp.userapi.com c848520 v848520411 11627...,0
35,"Возьмём как пример Россию, западноевропейские ...",0
...,...,...
14390,Принимал участие в этом опросе сказал что не д...,0
14392,"ТАСС, 21 марта. Премьер-министр Новой Зеландии...",0
14399,65 А чего так мало? Вот у гражданина был рейти...,0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0


### Threads

In [188]:
df_2ch_toxic_predicted = df_2ch.iloc[np.where(model_thr_cbc.predict(df_2ch_vec) == 1)]
df_2ch_untoxic_predicted = df_2ch.iloc[np.where(model_thr_cbc.predict(df_2ch_vec) == 0)]

In [189]:
df_2ch_toxic_predicted[df_2ch_toxic_predicted.toxic == 0]

,comment,toxic
468,Потому что он гомогеев убивал.\n,0
634,Сначала надо отрезать\n,0
1288,Ору!! Каммент огонь))),0
8031,Но поверх все равно изолентой намертво надо )\n,0


In [190]:
df_2ch_toxic_predicted[df_2ch_toxic_predicted.toxic == 1]

,comment,toxic
2,Собаке - собачья смерть\n,1
97,Что б он сам здох мразь\n,1
227,Правильно разбомбили сербских уродов. Жаль что...,1
348,стрелять в орущее быдло\n,1
373,Я атеист и армянин. Просто рот твой ебал Лучше...,1
...,...,...
13712,БРОСАЙ БЛЯТЬ Бросил. Пистолет выстрелил. Убил ...,1
13878,ВОЖДЬ Я УБЬЮ ТЕБЯ СУКА\n,1
13920,"Дебил блядь, стреляться в висок надо, полудуро...",1
14050,Сталинисты - наглухо отбитые животные.\n,1


# Saving models

In [130]:
from joblib import dump, load

In [ ]:
dump(models_pipeline["INS"], 'model_ins_logistic_2.joblib')
dump(models_pipeline["THR"], 'model_thr_logistic_2.joblib')
dump(models_pipeline["OBS"], 'model_obs_logistic_2.joblib')

['model_obs_logistic_2.joblib']

In [191]:
dump(model_ins_cbc, "model_ins_w2v_CBC.joblib")
dump(model_thr_cbc, "model_thr_w2v_CBC.joblib")
dump(model_obs_cbc, "model_obs_w2v_CBC.joblib")

['model_obs_w2v_CBC.joblib']